In [1]:
import sys 
import os

from shapely.geometry import mapping
from datetime import datetime

import numpy as np

In [2]:
rel_path = "../src"
directory_path = os.path.abspath(os.path.join(os.getcwd(), rel_path))
sys.path.append(directory_path)

from era5_weather import clean_column_names, get_weather_data_df, WeatherAPI
from nex_gddp_cmip6 import get_nex_dataset, TIME_OPTIMIZED_SCENARIOS, AVAILABLE_VARIABLES, load_region, select_region

C:\Users\manwi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
geodf = load_region('../assets/data/external/MI_avas.geojson')

# https://gis.stackexchange.com/questions/343725/convert-geojson-to-geopandas-geodataframe
geo_series = geodf.geometry

# init weather object
weather = WeatherAPI(geo_series, 'US/Central')

## Get Region Area

In [8]:
# https://www.freecodecamp.org/news/strftime-in-python/
start_date = datetime.fromisoformat("2022-01-01 00:00:00")
end_date = datetime.fromisoformat("2022-12-31 00:00:00")

# get area data - for Michigan Wines, took about 30 seconds.
weather_area_ds = weather._get_area_data(start_date = start_date, end_date = end_date, frequency = 'M', model = 'era5')

In [9]:
weather_area_ds 

<xarray.Dataset> Size: 159MB
Dimensions:                          (latitude: 19, longitude: 17, time: 8760)
Coordinates:
  * latitude                         (latitude) float32 76B 46.0 45.75 ... 41.5
  * longitude                        (longitude) float32 68B 93.0 93.25 ... 97.0
  * time                             (time) datetime64[ns] 70kB 2022-01-01 .....
Data variables: (12/14)
    temperature                      (time, latitude, longitude) float32 11MB ...
    dewpoint_temperature             (time, latitude, longitude) float32 11MB ...
    wind_speed                       (time, latitude, longitude) float32 11MB ...
    surface_pressure                 (time, latitude, longitude) float32 11MB ...
    surface_solar_radiation          (time, latitude, longitude) float32 11MB ...
    surface_thermal_radiation        (time, latitude, longitude) float32 11MB ...
    ...                               ...
    surface_diffuse_solar_radiation  (time, latitude, longitude) float32 11MB ...
    total_cloud_cover                (time, latitude, longitude) float32 11MB ...
    total_precipitation              (time, latitude, longitude) float32 11MB ...
    relative_humidity                (time, latitude, longitude) float32 11MB ...
    snowfall                         (time, latitude, longitude) float32 11MB ...
    snow_depth                       (time, latitude, longitude) float32 11MB ...
Attributes:
    Source:       ECMWF ERA5 reanalysis data - Copernicus Climate Change Serv...
    Conventions:  CF-1.6
    Institution:  Source data processed by Oikolab for cloud-optimized storag...
    model:        era5

In [10]:
weather_df = weather._create_dataframe(weather_area_ds)

In [12]:
weather_df.shape

(2829480, 21)

In [11]:
weather_df.head()

,lat,lon,timestamp_utc,temperature_degC,dewpoint_temperature_degC,wind_speed_m_per_s,surface_pressure_Pa,surface_solar_radiation_W_per_m2,surface_thermal_radiation_W_per_m2,total_solar_radiation,...,surface_diffuse_solar_radiation,total_cloud_cover,total_precipitation_mm_of_water_equivalent,relative_humidity_0_1,snowfall,snow_depth,model,timestamp_local,snowfall_mm_of_water_equivalent,snow_depth_mm_of_water_equivalent
0,46.0,93.0,2022-01-01 00:00:00+00:00,-20.933380,-28.039856,3.872967,77776.242188,0.000000,150.961288,0.000000,...,0.000000,0.414005,0.0,0.528548,0.0,43.031693,era5,2021-12-31 18:00:00-06:00,None,None
1,46.0,93.0,2022-01-01 01:00:00+00:00,-20.460144,-28.558197,3.429763,77775.367188,0.000000,152.670654,0.000000,...,0.000000,0.651031,0.0,0.483455,0.0,43.031693,era5,2021-12-31 19:00:00-06:00,None,None
2,46.0,93.0,2022-01-01 02:00:00+00:00,-17.464569,-27.870331,3.218068,77763.929688,7.061458,151.689987,3.302743,...,4.664861,0.899165,0.0,0.399079,0.0,43.031693,era5,2021-12-31 20:00:00-06:00,None,None
3,46.0,93.0,2022-01-01 03:00:00+00:00,-18.121582,-27.067596,2.971045,77759.531250,101.816666,154.094025,47.535484,...,42.440346,0.843834,0.0,0.454255,0.0,43.031693,era5,2021-12-31 21:00:00-06:00,None,None
4,46.0,93.0,2022-01-01 04:00:00+00:00,-18.131332,-25.001221,2.571847,77746.335938,224.700806,159.603577,104.460449,...,77.411842,0.804694,0.0,0.548667,0.0,43.031693,era5,2021-12-31 22:00:00-06:00,None,None


## function: _get_location_data

Notes: Many rows returned, need to double check validity

In [4]:
start_date = datetime.fromisoformat("2022-01-01 00:00:00")
end_date = datetime.fromisoformat("2022-12-31 00:00:00")

# for sample geojson, takes about 8 seconds
weather_location_df = weather._get_location_data(start_date = start_date, end_date = end_date, frequency = 'M', model = 'era5')

c:\Users\manwi\OneDrive\Python_Workbench\um-mads\src\era5_weather.py:186: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lat, lon = self.geometry.centroid.y.values, self.geometry.centroid.x.values


In [5]:
weather_location_df.shape

(43800, 18)

In [7]:
weather_location_df.head()

,"coordinates (lat,lon)",model (name),model elevation (surface),utc_offset (hrs),temperature (degC),dewpoint_temperature (degC),wind_speed (m/s),surface_pressure (Pa),surface_solar_radiation (W/m^2),surface_thermal_radiation (W/m^2),surface_net_solar_radiation (W/m^2),surface_direct_solar_radiation (W/m^2),surface_diffuse_solar_radiation (W/m^2),total_cloud_cover (0-1),total_precipitation (mm of water equivalent),relative_humidity (0-1),snowfall (mm of water equivalent),snow_depth (mm of water equivalent)
2022-01-01 00:00:00,"(45.35647528080625, 95.51675022214177)",era5,2075.11,7.0,-16.71,-27.93,0.81,79331.09,0.00,158.81,0.00,0.00,0.00,0.02,0.0,0.37,0.0,0.95
2022-01-01 01:00:00,"(45.35647528080625, 95.51675022214177)",era5,2075.11,7.0,-17.07,-27.15,1.12,79349.94,0.00,158.66,0.00,0.00,0.00,0.01,0.0,0.41,0.0,0.95
2022-01-01 02:00:00,"(45.35647528080625, 95.51675022214177)",era5,2075.11,7.0,-14.09,-25.03,1.25,79356.33,17.86,158.73,14.10,8.59,9.27,0.03,0.0,0.39,0.0,0.95
2022-01-01 03:00:00,"(45.35647528080625, 95.51675022214177)",era5,2075.11,7.0,-14.13,-25.22,1.38,79345.99,131.43,160.21,103.95,88.09,43.35,0.14,0.0,0.39,0.0,0.95
2022-01-01 04:00:00,"(45.35647528080625, 95.51675022214177)",era5,2075.11,7.0,-14.30,-25.84,1.74,79353.94,256.37,162.54,203.16,195.31,61.06,0.55,0.0,0.37,0.0,0.95


In [16]:
# checking shape
dyrus = weather.geometry.to_crs("EPSG:4326")
lat, lon = dyrus.centroid.y.values, dyrus.centroid.x.values
print(f"lat: {lat}, lon: {lon}")     

lat: [45.35647528 42.16207585 44.9149431  42.54014048 44.89079311], lon: [95.51675022 93.89195281 94.20960754 93.83252237 94.47556306]


C:\Users\manwi\AppData\Local\Temp\ipykernel_55852\2709532432.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lat, lon = dyrus.centroid.y.values, dyrus.centroid.x.values


In [17]:
oddone = weather.geometry
lat, lon = oddone.centroid.y.values, oddone.centroid.x.values
print(f"lat: {lat}, lon: {lon}")

lat: [45.35647528 42.16207585 44.9149431  42.54014048 44.89079311], lon: [95.51675022 93.89195281 94.20960754 93.83252237 94.47556306]


C:\Users\manwi\AppData\Local\Temp\ipykernel_55852\3097027570.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lat, lon = oddone.centroid.y.values, oddone.centroid.x.values
